# Introducción.

La Encuesta Nacional sobre Servicios Financieros y Ahorro (ENSAFI) de 2023 amplía el amrco sobre la situación financiera.

Enfocándose en el uso de servicios financieros y las prácticas de ahorro.

ENSAFI se centra en evaluar cómo la población maneja sus ingresos y gastos, incluyendo el uso de cuentas de ahorro, tarjetas de crédito y otras formas de crédito.

Además, explora las acciones que las personas toman cuando no pueden cubrir sus gastos, como préstamos a familiares, uso de ahorros, y reducción de gastos. Los resultados de ENSAFI ofrecen una perspectiva actualizada sobre las tendencias financieras y las dificultades que enfrentan los mexicanos en su vida diaria.

Esta vez, cond atos más completos. Se buscara comprobar origenes de ciertos datos y al mismo tiempo visualizar los indices de **Edcuación Financiera**.

In [32]:
pip install pywaffle

In [33]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt
from pywaffle import Waffle

import warnings
warnings.simplefilter('ignore')

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
def leer_catalogo(catalog_folder, filename, encoding='utf-8'):

    catalog_path = os.path.join(catalog_folder, filename)
    df_catalog = pd.read_csv(catalog_path, encoding=encoding)
    return pd.Series(df_catalog.descrip.values, index=df_catalog.cve).to_dict()

def aplicar_mapeo(df, mapeos):

    for col, mapping in mapeos.items():
        if col in df.columns:
            df[col] = df[col].replace(mapping)
    return df

file_path = '/content/drive/MyDrive/FINANZAS/ENSAFI/conjunto_de_datos_tmodulo_ensafi_2023.csv'
catalog_folder = '/content/drive/MyDrive/FINANZAS/ENSAFI/catalogos'

columnas_de_interes = [
    'P5_14', 'P5_19', 'P5_19A',
    'P7_1', 'P7_2_1', 'P7_2_2',
    'P7_5_1', 'P7_5_2', 'P7_5_3', 'P7_5_4', 'P7_5_5', 'P7_5_6',
    'P6_9',
    'P6_10_1', 'P6_10_2', 'P6_10_3', 'P6_10_4', 'P6_10_5', 'P6_10_6', 'P6_10_7',
    'P7_12_1', 'P7_12_2', 'P7_12_3', 'P7_12_4', 'P7_12_5', 'P7_12_6',
    'P7_7_5', 'P7_7_6',
    'P7_8_1', 'P7_8_2', 'P7_8_3', 'P7_8_4', 'P7_8_5', 'P7_8_6',
    'P6_2_04', 'P6_2_06', 'P6_2_07', 'P6_3',
    'EDAD_V', 'SEXO'
]

df_General = pd.read_csv(file_path, usecols=columnas_de_interes, encoding='utf-8')

mapeos = {}

for filename in os.listdir(catalog_folder):
    if filename.endswith('.csv'):
        column_name = filename.split('.')[0]
        mapeos[column_name] = leer_catalogo(catalog_folder, filename, encoding='utf-8')

df_General = aplicar_mapeo(df_General, mapeos)

df_General.head()


,EDAD_V,P5_14,P5_19,P5_19A,P6_2_04,P6_2_06,P6_2_07,P6_3,P6_9,P6_10_1,...,P7_8_4,P7_8_5,P7_8_6,P7_12_1,P7_12_2,P7_12_3,P7_12_4,P7_12_5,P7_12_6,SEXO
0,65,Se dedica a los quehaceres del hogar o al cuid...,NaN,NaN,No,No,No,NaN,No,No,...,Muy bien,Muy bien,Nada,Sí,Sí,No,No,Sí,No,Mujer
1,32,Trabajó por lo menos una hora,12000.0,3.0,Sí,No,No,Sí,No,Sí,...,Nada,Algo,Nada,No,Sí,No,Sí,No,No,Mujer
2,34,Trabajó por lo menos una hora,1800.0,1.0,No,No,No,Sí,No,No,...,Nada,Algo,Nada,Sí,No,No,No,Sí,No,Mujer
3,85,Se dedica a los quehaceres del hogar o al cuid...,NaN,NaN,No,No,No,Sí,Sí,NaN,...,Muy bien,Muy bien,Muy bien,No,No,No,No,No,No,Mujer
4,30,Trabajó por lo menos una hora,6000.0,2.0,No,No,No,Sí,No,No,...,Muy bien,Muy bien,Algo,Sí,Sí,No,Sí,Sí,No,Mujer


In [36]:
def aplicar_multiplicador(row):
    if row['P5_19'] == 99999:
        return row['P5_19']
    elif row['P5_19A'] == 1:
        ingreso_normalizado = row['P5_19'] * 4
    elif row['P5_19A'] == 2:
        ingreso_normalizado = row['P5_19'] * 2
    elif row['P5_19A'] == 3:
        ingreso_normalizado = row['P5_19']
    elif row['P5_19A'] == 4:
        ingreso_normalizado = row['P5_19'] / 12
    else:
        ingreso_normalizado = row['P5_19']

    if ingreso_normalizado > 98000:
        return 98000
    else:
        return ingreso_normalizado

df_General['P5_19'] = df_General.apply(aplicar_multiplicador, axis=1)

df_General.drop(columns=['P5_19A'], inplace=True)

In [37]:
df_General.head()

,EDAD_V,P5_14,P5_19,P6_2_04,P6_2_06,P6_2_07,P6_3,P6_9,P6_10_1,P6_10_2,...,P7_8_4,P7_8_5,P7_8_6,P7_12_1,P7_12_2,P7_12_3,P7_12_4,P7_12_5,P7_12_6,SEXO
0,65,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,NaN,No,No,No,...,Muy bien,Muy bien,Nada,Sí,Sí,No,No,Sí,No,Mujer
1,32,Trabajó por lo menos una hora,12000.0,Sí,No,No,Sí,No,Sí,No,...,Nada,Algo,Nada,No,Sí,No,Sí,No,No,Mujer
2,34,Trabajó por lo menos una hora,7200.0,No,No,No,Sí,No,No,No,...,Nada,Algo,Nada,Sí,No,No,No,Sí,No,Mujer
3,85,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,Sí,Sí,NaN,NaN,...,Muy bien,Muy bien,Muy bien,No,No,No,No,No,No,Mujer
4,30,Trabajó por lo menos una hora,12000.0,No,No,No,Sí,No,No,Sí,...,Muy bien,Muy bien,Algo,Sí,Sí,No,Sí,Sí,No,Mujer


# Descripción

| Columna          | Descripción |
|------------------|-------------|
| `P5_14`           | Actividad de la semana pasada |
| `P5_19`          | ¿Cuánto gana o recibe usted por trabajar (su actividad)? |
| `P7_1`           | ¿Usted lleva un registro de sus ingresos y gastos? |
| `P7_2_1`         | ¿Usted hace anotaciones de los gastos? |
| `P7_2_2`         | ¿Usted mantiene el dinero para pagos o deudas separados del dinero del gasto diario? |
| PREGUNTA         | **¿Por qué no tiene anotaciones o registro de sus ingresos y gastos?** |
| `P7_5_1`         | NO NECESITA HACER UN REGISTRO |
| `P7_5_2`         | NO TIENE TIEMPO |
| `P7_5_3`         | LE PARECE ABURRIDO |
| `P7_5_4`         | NO LO HABÍA PENSADO |
| `P7_5_5`         | NO SABE CÓMO HACERLO |
| `P7_5_6`         | OTRO MOTIVO |
| PREGUNTA         | **Manejo de Deuda** |
| `P6_9`           | Durante el Último mes, ¿lo que ganó o recibió fue suficiente para cubrir sus gastos sin endeudarse? |
| PREGUNTA         | **Durante el Último mes, para cubrir sus gastos...** |
| `P6_10_1`        | ¿Usted pidió prestado a familiares o amistades? |
| `P6_10_2`        | ¿Usted utilizó el dinero que tenía ahorrado? |
| `P6_10_3`        | ¿Usted redujo sus gastos? |
| `P6_10_4`        | ¿Usted vendió o empeñó algún bien? |
| `P6_10_5`        | ¿Usted solicitó un adelanto salarial, trabajó horas extras o hizo trabajo temporal? |
| `P6_10_6`        | ¿Usted utilizó su tarjeta de crédito o solicitó un crédito en un banco, institución financiera o de tienda departamental? |
| `P6_10_7`        | ¿Usted se atrasó en el pago de algún crédito o préstamo? |
| PREGUNTA         | **Educación Temprana** |
| `P7_12_1`        | Cuando usted era niña(o) ¿sus padres o tutores hablaban sobre la importancia del manejo adecuado del dinero? |
| `P7_12_2`        | Cuando usted era niña(o) ¿sus padres o tutores hablaban sobre la importancia del ahorro? |
| `P7_12_3`        | Cuando usted era niña(o) ¿había alguna actividad que le enseñara cómo manejar el dinero? |
| `P7_12_4`        | En la adolescencia, sus padres o tutores ¿le daban dinero (mesada) para sus gastos y usted los administraba? |
| `P7_12_5`        | En la adolescencia, sus padres o tutores ¿le hablaron sobre gastos necesarios e innecesarios? |
| `P7_12_6`        | En algún momento de su vida, ¿ha recibido algún curso para el registro de sus ingresos y gastos? |
| PREGUNTA         | **Frecuencia de malas decisiones** |
| `P7_7_5`         | ¿Con qué frecuencia se queda corta(o) de dinero porque gasta de más? |
| `P7_7_6`         | ¿Con qué frecuencia compra cosas a pesar de no poder pagarlas? |
| PREGUNTA         | **En qué medida lo describen las siguientes frases...** |
| `P7_8_1`         | Puede hacer frente a un gasto imprevisto importante |
| `P7_8_2`         | Está asegurando su futuro financiero |
| `P7_8_3`         | Dada su situación financiera, siente que tendrá las cosas que desea |
| `P7_8_4`         | Puede disfrutar la vida debido a la manera en que maneja su dinero |
| `P7_8_5`         | Le alcanza bien el dinero para cubrir sus gastos |
| `P7_8_6`         | Se siente tranquila(o) de que su dinero ahorrado sea suficiente |
| PREGUNTA         | **Ahorros e inversiones** |
| `P6_2_04`        | ¿Usted tiene cuenta de ahorro? |
| `P6_2_06`        | ¿Usted tiene CETES o depósito a plazo fijo (solo puede retirar en determinadas fechas)? |
| `P6_2_07`        | ¿Usted tiene fondo de inversión (tener acciones en casa de bolsa)? |
| `P6_3`           | Actualmente, ¿usted tiene ahorros en alguna de esas cuentas que mencionó? |
| `EDAD_V`         | Edad del entrevistado |
| `SEXO`           | Sexo del entrevistado |


# Código

In [38]:
nombres_columnas_enafi = {
    'P5_14': 'Actividad de la semana pasada',
    'P5_19': 'Ingreso por trabajo',
    'P7_1': 'Registro de ingresos y gastos',
    'P7_2_1': 'Anotaciones de gastos',
    'P7_2_2': 'Dinero separado para pagos y deudas',
    'P7_5_1': 'No necesita registro',
    'P7_5_2': 'No tiene tiempo',
    'P7_5_3': 'Le parece aburrido',
    'P7_5_4': 'No lo había pensado',
    'P7_5_5': 'No sabe cómo hacerlo',
    'P7_5_6': 'Otro motivo',
    'P6_9': 'Ingreso suficiente sin endeudarse',
    'P6_10_1': 'Préstamos a familiares o amistades',
    'P6_10_2': 'Uso de ahorros',
    'P6_10_3': 'Reducción de gastos',
    'P6_10_4': 'Venta o empeño de bienes',
    'P6_10_5': 'Adelanto salarial/trabajo extra',
    'P6_10_6': 'Uso de tarjeta de crédito o préstamo bancario',
    'P6_10_7': 'Atraso en pago de crédito o préstamo',
    'P7_12_1': 'Importancia del manejo del dinero en infancia',
    'P7_12_2': 'Importancia del ahorro en infancia',
    'P7_12_3': 'Actividades sobre manejo del dinero en infancia',
    'P7_12_4': 'Mesada en adolescencia',
    'P7_12_5': 'Habla sobre gastos necesarios e innecesarios en adolescencia',
    'P7_12_6': 'Curso sobre registro de ingresos y gastos',
    'P7_7_5': 'Frecuencia de quedarse sin dinero por gastar de más',
    'P7_7_6': 'Frecuencia de comprar cosas sin poder pagarlas',
    'P7_8_1': 'Capacidad para afrontar gastos imprevistos',
    'P7_8_2': 'Asegurando futuro financiero',
    'P7_8_3': 'Confianza en obtener lo deseado',
    'P7_8_4': 'Disfrute de la vida por buen manejo del dinero',
    'P7_8_5': 'Dinero suficiente para gastos',
    'P7_8_6': 'Tranquilidad por dinero ahorrado suficiente',
    'P6_2_04': 'Cuenta de ahorro',
    'P6_2_06': 'CETES o depósito a plazo fijo',
    'P6_2_07': 'Fondo de inversión',
    'P6_3': 'Ahorros en cuentas mencionadas',
    'EDAD_V': 'Edad',
    'SEXO': 'Sexo'
}

df_General.rename(columns=nombres_columnas_enafi, inplace=True)

df_General


,Edad,Actividad de la semana pasada,Ingreso por trabajo,Cuenta de ahorro,CETES o depósito a plazo fijo,Fondo de inversión,Ahorros en cuentas mencionadas,Ingreso suficiente sin endeudarse,Préstamos a familiares o amistades,Uso de ahorros,...,Disfrute de la vida por buen manejo del dinero,Dinero suficiente para gastos,Tranquilidad por dinero ahorrado suficiente,Importancia del manejo del dinero en infancia,Importancia del ahorro en infancia,Actividades sobre manejo del dinero en infancia,Mesada en adolescencia,Habla sobre gastos necesarios e innecesarios en adolescencia,Curso sobre registro de ingresos y gastos,Sexo
0,65,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,NaN,No,No,No,...,Muy bien,Muy bien,Nada,Sí,Sí,No,No,Sí,No,Mujer
1,32,Trabajó por lo menos una hora,12000.0,Sí,No,No,Sí,No,Sí,No,...,Nada,Algo,Nada,No,Sí,No,Sí,No,No,Mujer
2,34,Trabajó por lo menos una hora,7200.0,No,No,No,Sí,No,No,No,...,Nada,Algo,Nada,Sí,No,No,No,Sí,No,Mujer
3,85,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,Sí,Sí,NaN,NaN,...,Muy bien,Muy bien,Muy bien,No,No,No,No,No,No,Mujer
4,30,Trabajó por lo menos una hora,12000.0,No,No,No,Sí,No,No,Sí,...,Muy bien,Muy bien,Algo,Sí,Sí,No,Sí,Sí,No,Mujer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20443,58,Trabajó por lo menos una hora,8000.0,No,No,No,NaN,No,Sí,No,...,Nada,Poco,Nada,No,No,No,No,No,No,Mujer
20444,60,Estaba en otra situación diferente a las anter...,NaN,No,No,No,NaN,No,No,Sí,...,Poco,Nada,Poco,No,Sí,No,No,Sí,No,Hombre
20445,69,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,No,Sí,NaN,NaN,...,Algo,Algo,Poco,Sí,Sí,No,No,Sí,No,Mujer
20446,81,Se dedica a los quehaceres del hogar o al cuid...,NaN,No,No,No,No,No,No,Sí,...,Poco,Algo,Poco,No,No,No,No,No,No,Mujer


In [39]:
df_General.isnull().sum()

,0
Edad,0
Actividad de la semana pasada,0
Ingreso por trabajo,6108
Cuenta de ahorro,0
CETES o depósito a plazo fijo,0
Fondo de inversión,0
Ahorros en cuentas mencionadas,7994
Ingreso suficiente sin endeudarse,0
Préstamos a familiares o amistades,14224
Uso de ahorros,14224


In [40]:
df_General.to_csv('/content/drive/MyDrive/FINANZAS/ENSAFI/df_General_ENSAFI.csv', index=False)

# Distribución de Edad y Sexo

In [41]:
conteo_sexo = df_General['Sexo'].value_counts()

fig = px.pie(conteo_sexo,
             names=conteo_sexo.index,
             values=conteo_sexo.values,
             hole=.3,
             title='Distribución de Hombres y Mujeres en la encuesta',
             color_discrete_sequence=px.colors.qualitative.D3)

fig.show()

In [42]:
bins = [18, 30, 50, 70, float('inf')]
labels = ['18-30', '31-50', '51-70', '71+']

df_General['GRUPO_EDAD'] = pd.cut(df_General['Edad'], bins=bins, labels=labels, right=False)

conteo_grupos = df_General['GRUPO_EDAD'].value_counts().sort_index()

total_personas = df_General.shape[0]

conteo_grupos['Total'] = total_personas

fig_barras = px.bar(conteo_grupos,
                    color=conteo_grupos.index,
                    color_discrete_sequence=px.colors.qualitative.D3,
                    title='Distribución de Edad de los Encuestados',
                    labels={'index': 'Grupo de Edad', 'value': 'Cantidad'},
                    text_auto=True)
fig_barras.show()


# Ingresos y egresos (Uso primerizo del Dinero)

Se contempla el analisis estadistico del **Modo y Cantidad de ingresos** Así como del uso que se les da.

In [43]:

conteo_actividad = df_General['Actividad de la semana pasada'].value_counts()

fig_actividad = px.bar(
    conteo_actividad,
    x=conteo_actividad.index,
    y=conteo_actividad.values,
    color=conteo_actividad.index,
    color_discrete_sequence=px.colors.qualitative.D3,
    title='Actividad de la Semana Pasada',
    labels={'x': 'Actividad', 'y': 'Cantidad'},
    text_auto=True
)

fig_actividad.show()


In [44]:
bins = [0, 1, 1000, 5000, 10000, 20000, 30000, 40000, 50000, 98000, 98888, float('inf')]
labels = ['No recibe ingresos', '0-1k', '+1k', '+5k', '+10k', '+20k', '+30k', '+40k', '+50k', '98k o más', 'No responde']

df_General['Grupo_Ingreso'] = pd.cut(df_General['Ingreso por trabajo'], bins=bins, labels=labels, right=False, include_lowest=True)

conteo_grupos_ingreso = df_General['Grupo_Ingreso'].value_counts().sort_index()

fig_barras = px.bar(
    conteo_grupos_ingreso,
    x=conteo_grupos_ingreso.index,
    y=conteo_grupos_ingreso.values,
    color=conteo_grupos_ingreso.index,
    color_discrete_sequence=px.colors.qualitative.D3,
    title='Distribución de Ingresos por Actividad',
    labels={'x': 'Grupo de Ingreso', 'y': 'Cantidad'},
    text_auto=True
)

fig_barras.show()


Un **Sector grande** de la población se encuentra en un rango de los **Cinco Mil Pesos Mexicanos**

Seguido de **Mil Pesos** y **Diez Mil Pesos** Una medía bastante alentadora en cuanto a promedio.

In [45]:
conteo_registro = df_General['Registro de ingresos y gastos'].value_counts()
conteo_anotaciones = df_General['Anotaciones de gastos'].value_counts()
conteo_dinero_separado = df_General['Dinero separado para pagos y deudas'].value_counts()

fig = sp.make_subplots(rows=1, cols=3, subplot_titles=(
    "Registro de ingresos y gastos",
    "Anotaciones de gastos",
    "Dinero separado para pagos y deudas"
), specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=conteo_registro.index, values=conteo_registro.values, name="Registro de ingresos y gastos", hole=.3, marker=dict(colors=px.colors.qualitative.D3)), 1, 1)

fig.add_trace(go.Pie(labels=conteo_anotaciones.index, values=conteo_anotaciones.values, name="Anotaciones de gastos", hole=.3, marker=dict(colors=px.colors.qualitative.D3)), 1, 2)

fig.add_trace(go.Pie(labels=conteo_dinero_separado.index, values=conteo_dinero_separado.values, name="Dinero separado para pagos y deudas", hole=.3, marker=dict(colors=px.colors.qualitative.D3)), 1, 3)

fig.update_layout(title_text="Realiza alguna de las siguientes acciones...")

fig.show()


Un sector **muy considerable de la población** No hace un registro de sus Ingesos y Gastos.

Tampoco hace **Anotaciones de Gastos**

Y vemos de **Forma ngeativa** que un sector **Parcialmente mayor** NO divide sus Deudas de Dinero Utilizable.

Como ya habiamos mencionado en la Encuesta anterior **No tener un registro presente de Gastos y Deudas** da la libertad a **fuga de ingresos** que en muhcas ocasciones el sector de la población afectado **Desconoce**.

A la par, anteriormente se había especulado sobre el motivo de esta toma de decisiones, **Suponiendo** que sería ante una **falta de conocimiento de estos registros**

o a un sentimiento de **excepticismo a la necesidad de estas**

In [46]:
columnas_motivos = [
    'No necesita registro',
    'No tiene tiempo',
    'Le parece aburrido',
    'No lo había pensado',
    'No sabe cómo hacerlo',
    'Otro motivo'
]
conteo_motivos = {columna: df_General[columna].value_counts().get('Sí se mencionó', 0) for columna in columnas_motivos}

df_motivos = pd.DataFrame(list(conteo_motivos.items()), columns=['Motivo', 'Cantidad'])

fig_barras = px.bar(
    df_motivos,
    x='Motivo',
    y='Cantidad',
    color='Motivo',
    color_discrete_sequence=px.colors.qualitative.D3,
    title='Motivos para No Registrar Ingresos y Gastos',
    labels={'Cantidad': 'Número de Personas', 'Motivo': 'Motivo'},
    text_auto=True
)

fig_barras.show()


In [47]:
columnas_motivos = [
    'No necesita registro',
    'No tiene tiempo',
    'Le parece aburrido',
    'No lo había pensado',
    'No sabe cómo hacerlo',
    'Otro motivo'
]

df_motivos_edad = df_General.melt(id_vars=['GRUPO_EDAD'], value_vars=columnas_motivos,
                                  var_name='Motivo', value_name='Respuesta')

df_motivos_edad = df_motivos_edad[df_motivos_edad['Respuesta'] == 'Sí se mencionó']

conteo_motivos_edad = df_motivos_edad.groupby(['GRUPO_EDAD', 'Motivo']).size().reset_index(name='Cantidad')

fig_motivos_edad = px.bar(
    conteo_motivos_edad,
    x='GRUPO_EDAD',
    y='Cantidad',
    color='Motivo',
    title='Relación entre Edad y Motivos para No Registrar Ingresos y Gastos',
    labels={'GRUPO_EDAD': 'Grupo de Edad', 'Cantidad': 'Número de Personas', 'Motivo': 'Motivo'},
    color_discrete_sequence=px.colors.qualitative.D3
)

fig_motivos_edad.update_layout(barmode='stack')

fig_motivos_edad.show()


Podemos **Resaltar algúnas cosas bastante improtantes en este lugar**

- Un sector **Muy Grande** Afirma **No necesitar de un registro** cosa que se vuelve predominante en todas las Edades.

- El sector **Adulto** es el que mayor cantidad de personas que afirman **No tener tiempo para un registro**

Cosa que puede verse relacionada con **La energía y tiempo de trabajo que invierten** siendo ese un rango de **Edad donde se suelen buscar ingresos constantes**

Este grupo, probablemente compuesto por personas en plena carrera profesional y con responsabilidades familiares, podría beneficiarse significativamente de métodos más eficientes para gestionar sus finanzas.

- **Entre más Edad** aumenta la cantidad de gente que **No sabe como hacer un registro** lo que indicaría un **desconocimiento o ausencia de Educación financiera en adultos mayores**

Observando detenidamente.

# Dinero Prestado

In [48]:
conteo_ingreso_suficiente = df_General['Ingreso suficiente sin endeudarse'].value_counts()

fig_pastel = px.pie(
    conteo_ingreso_suficiente,
    names=conteo_ingreso_suficiente.index,
    values=conteo_ingreso_suficiente.values,
    title='Durante el Último mes, ¿lo que ganó o recibió fue suficiente para cubrir sus gastos sin endeudarse?',
    color_discrete_sequence=px.colors.qualitative.D3,
    labels={'names': 'Respuesta', 'values': 'Cantidad'},
    hole=0.3
)

fig_pastel.update_traces(textposition='inside', textinfo='percent+label')

fig_pastel.show()


**Mayoritariamente** La población prefiere Reducir sus Gastos.

**Sin emabrg** tambien vemos una cercania con aquellos que prefieren **Pedir dinero a amistades o Familiares**

---

# Educación Financiera Temprana

In [49]:
columnas_infancia = [
    'Importancia del manejo del dinero en infancia',
    'Importancia del ahorro en infancia',
    'Actividades sobre manejo del dinero en infancia'
]

conteo_infancia = {columna: df_General[columna].value_counts() for columna in columnas_infancia}

fig_infancia = sp.make_subplots(rows=1, cols=3, subplot_titles=columnas_infancia, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])

for i, columna in enumerate(columnas_infancia):
    fig_infancia.add_trace(
        go.Pie(
            labels=conteo_infancia[columna].index,
            values=conteo_infancia[columna].values,
            name=columna,
            hole=0.3,
            marker=dict(colors=px.colors.qualitative.D3)
        ),
        1, i+1
    )

fig_infancia.update_layout(
    title_text='Cuando usted era niña(o) ¿sus padres o tutores hablaban sobre...'
)

fig_infancia.show()


Podemos notar que en grandes rasgos. **Un sector mayoritario** de la población.

**No recibio educación Financiera de calidad** en su niñez

In [50]:
columnas_adolescencia = [
    'Mesada en adolescencia',
    'Habla sobre gastos necesarios e innecesarios en adolescencia'
]

conteo_adolescencia = {columna: df_General[columna].value_counts() for columna in columnas_adolescencia}

fig_adolescencia = sp.make_subplots(rows=1, cols=2, subplot_titles=columnas_adolescencia, specs=[[{'type':'domain'}, {'type':'domain'}]])

for i, columna in enumerate(columnas_adolescencia):
    fig_adolescencia.add_trace(
        go.Pie(
            labels=conteo_adolescencia[columna].index,
            values=conteo_adolescencia[columna].values,
            name=columna,
            hole=0.3,
            marker=dict(colors=px.colors.qualitative.D3)
        ),
        1, i+1
    )

fig_adolescencia.update_layout(
    title_text='En la adolescencia, sus padres o tutores le proprcionarón algo de lo siguiente:'
)

fig_adolescencia.show()


Se repie un patrón en la **Ausencia de Responsabilidad Financiera Temprana**

Así como de la **Concientización de los Tipos de Gastos**

In [51]:
conteo_curso = df_General['Curso sobre registro de ingresos y gastos'].value_counts()

fig_curso = px.pie(
    conteo_curso,
    names=conteo_curso.index,
    values=conteo_curso.values,
    title='¿Ha tomado algún curso sobre registro de ingresos y gastos?',
    color_discrete_sequence=px.colors.qualitative.D3,
    labels={'names': 'Respuesta', 'values': 'Cantidad'},
    hole=0.3
)

fig_curso.update_traces(textposition='inside', textinfo='percent+label')

fig_curso.show()


In [52]:
conteo_curso_edad = df_General.groupby(['GRUPO_EDAD', 'Curso sobre registro de ingresos y gastos']).size().reset_index(name='Cantidad')

fig_curso_edad = px.bar(
    conteo_curso_edad,
    x='GRUPO_EDAD',
    y='Cantidad',
    color='Curso sobre registro de ingresos y gastos',
    title='Relación entre Edad y Curso sobre Registro de Ingresos y Gastos',
    labels={'GRUPO_EDAD': 'Grupo de Edad', 'Cantidad': 'Número de Personas', 'Curso sobre registro de ingresos y gastos': 'Participación en Curso'},
    color_discrete_sequence=px.colors.qualitative.D3
)

fig_curso_edad.update_layout(barmode='stack')

fig_curso_edad.show()


**86.4%** de la Población **NO ha tomado un Curso referente a Finanzas**

Paralelamente, **Un grupo de edad Adulto** es el que **Menos personas han tomado un curso** y al mismo tiempo **Los que más han tomado cursos**

Esto puede reflejar una **creciente necesidad** de un mejor manejo de dinero por parte de las generaciones adultas.

Una busqueda de la sostenibilidas de sus ingresos que busca alternativas.

# Frecuencia de Malas Decisiones

In [53]:
df_General['Frecuencia de quedarse sin dinero por gastar de más'].value_counts()

,count
Frecuencia de quedarse sin dinero por gastar de más,
A veces,7073
Casi nunca,4896
Nunca,4563
Casi siempre,2170
Siempre,1746


In [54]:
df_General['Frecuencia de comprar cosas sin poder pagarlas'].value_counts()

,count
Frecuencia de comprar cosas sin poder pagarlas,
Nunca,13301
Casi nunca,3913
A veces,2272
Casi siempre,560
Siempre,402


In [55]:
conteo_quedarse_sin_dinero = df_General['Frecuencia de quedarse sin dinero por gastar de más'].value_counts().reindex(['Siempre', 'Casi siempre', 'A veces', 'Casi nunca', 'Nunca'])
conteo_comprar_sin_poder_pagar = df_General['Frecuencia de comprar cosas sin poder pagarlas'].value_counts().reindex(['Siempre', 'Casi siempre', 'A veces', 'Casi nunca', 'Nunca'])

fig_quedarse_sin_dinero = px.bar(
    conteo_quedarse_sin_dinero,
    x=conteo_quedarse_sin_dinero.index,
    y=conteo_quedarse_sin_dinero.values,
    title='Frecuencia de Quedarse sin Dinero por Gastar de Más',
    labels={'x': 'Frecuencia', 'y': 'Cantidad'},
    color=conteo_quedarse_sin_dinero.index,
    color_discrete_sequence=px.colors.qualitative.D3
)

fig_quedarse_sin_dinero.update_layout(xaxis_title='Frecuencia', yaxis_title='Cantidad', showlegend=False)

fig_comprar_sin_poder_pagar = px.bar(
    conteo_comprar_sin_poder_pagar,
    x=conteo_comprar_sin_poder_pagar.index,
    y=conteo_comprar_sin_poder_pagar.values,
    title='Frecuencia de Comprar Cosas sin Poder Pagarlas',
    labels={'x': 'Frecuencia', 'y': 'Cantidad'},
    color=conteo_comprar_sin_poder_pagar.index,
    color_discrete_sequence=px.colors.qualitative.D3
)

fig_comprar_sin_poder_pagar.update_layout(xaxis_title='Frecuencia', yaxis_title='Cantidad', showlegend=False)

fig_quedarse_sin_dinero.show()
fig_comprar_sin_poder_pagar.show()


Una **suposición simple** sería que estas graficas se relacionan con los **registros de Gastos e Ingresos**

El hecho de que **Gran parte del sector poblaciona** suela **quedarse sin dinero por gastar de más**

Pero que en contraste, **Nunca compren cosas que no puedan pagar** se vuelve inconsistente una con otra.

Esto podría deverse a multiples razones.

- No se hace un correcto manejo de presupuestos, derivanod en compras no planeadas que desestabilizan la economia de la gente a futuro.

- Cultura de Crédito, donde se prefiere comprar las cosas hoy e irlas pasgando despues sin considerar gastos a futuro.

La priemra de estas opciones se vuelve la más pausible ante los graficos vistos con anterioridad en este NoteBook. Considerando que la gestión en finanzas de un gran sector de la población se ve regido por el momento, y no por una planeación rigurosa.

In [56]:
columnas_frases = [
    'Capacidad para afrontar gastos imprevistos',
    'Asegurando futuro financiero',
    'Confianza en obtener lo deseado',
    'Disfrute de la vida por buen manejo del dinero',
    'Dinero suficiente para gastos',
    'Tranquilidad por dinero ahorrado suficiente'
]

fig = sp.make_subplots(rows=2, cols=3, subplot_titles=columnas_frases)

colores = px.colors.qualitative.D3

for i, columna in enumerate(columnas_frases):
    conteo_respuestas = df_General[columna].value_counts().reindex(['Completamente', 'Muy bien', 'Algo', 'Poco', 'Nada']).fillna(0)

    fig.add_trace(
        go.Bar(
            x=conteo_respuestas.index,
            y=conteo_respuestas.values,
            name=columna,
            marker=dict(color=colores[i % len(colores)])
        ),
        row=(i // 3) + 1, col=(i % 3) + 1
    )

fig.update_layout(
    title_text="En qué medida te describen las siguientes frases",
    showlegend=False,
    height=800,
    width=1600
)

fig.show()


De forma directa, se contempla que el sector **Tiene dinero para solventar etapas dificiles que se aproximen o vivir comodamente el futuro cercano**

Pero **referente al futuro a Largo Plazo** se ve una diminución en la gente que **afirma estar preparada para el futuro, o tener suficientes ahorros**

# Cuentas de Ahorro

In [57]:
columnas_cuentas = [
    'Cuenta de ahorro',
    'CETES o depósito a plazo fijo',
    'Fondo de inversión'
]

conteo_cuentas = {columna: df_General[columna].value_counts() for columna in columnas_cuentas}

fig_cuentas = sp.make_subplots(rows=1, cols=3, subplot_titles=columnas_cuentas, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])

for i, columna in enumerate(columnas_cuentas):
    fig_cuentas.add_trace(
        go.Pie(
            labels=conteo_cuentas[columna].index,
            values=conteo_cuentas[columna].values,
            name=columna,
            hole=0.3,
            marker=dict(colors=px.colors.qualitative.D3)
        ),
        1, i+1
    )

fig_cuentas.update_layout(
    title_text='¿Tienes alguna de estas cuentas?'
)

fig_cuentas.show()


**un sector muy corto de la población** cuenta con formas de ahorro a largo plazo.

Considerandoq ue estas son las 3 más conocidas en un sector publico **Y las más accesibles** resulta en un porcentaje **Muy bajo**

In [58]:
import pandas as pd
import plotly.express as px

conteo_ahorros = df_General['Ahorros en cuentas mencionadas'].value_counts().reindex(['Sí', 'No']).reset_index()
conteo_ahorros.columns = ['Respuesta', 'Cantidad']

fig_ahorros = px.pie(
    conteo_ahorros,
    names='Respuesta',
    values='Cantidad',
    title='¿Tienes ahorros en cuentas mencionadas?',
    color_discrete_sequence=px.colors.qualitative.D3,
    hole=0.3
)

fig_ahorros.update_traces(textposition='inside', textinfo='percent+label')

fig_ahorros.update_layout(margin=dict(t=80))

fig_ahorros.show()


Adentrando en ese **Pequeño porcentaje que si tiene algun tipo de cuenta**

**Unicamente el 40.8%** tiene dinero invertido/agregado en dichas cuentas.

Lo cual **Reduce** mucho más la cantidad del sector que iniverte sus ingresos a futuro.

# Conclusion

Con más a datos a disposición, se busco explorar la **eduación Financiera**.

Encontrando que la forma tangible de este no se ha sedimentado por completo.

Se ven ciertas intenciones ante la busqueda de metodos para ahorro o ingresos extras que requieran de menor esfuerzo.

Pero la aplciación a la paractica se ve mermada una vez se realiza, pues muchas de las acciones necesarias (invertir, ahorrar, hacer registros) se dejan de lado o simplemente se dejan al olvido.

Una posible causa (especulación sin fundamentos) sea un **sesgo** cultura, donde a diferencia de otras naciones, la cultura del dinero y el ahorro no se ha visto tan desarrollado en la población. Viviendo muchas veces al día.

Estadisticamente se puede comprender que no hay un gran conocimiento de *eduación financiera*. Sin embargo siempre es importante considerar que gran parte del pueblo mexicano vive con sueldos ajustados y deudas ya existentes.

Factores que en muhcas ocasciones, impiden un correcto crecimiento economico.

In [59]:
df_General.to_csv('/content/drive/MyDrive/FINANZAS/ENSAFI/df_General_ENSAFI.csv', index=False)